In [ ]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.preprocessing import LabelEncoder



import pickle

import pandas as pd
import numpy as np


from sklearn.base import TransformerMixin

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection


from sklearn.ensemble import RandomForestClassifier

from konlpy.tag import MeCab
mecab = Mecab()

In [ ]:
train = pd.read_csv('train.csv')
train_text = train['overview'].to_list()
test = pd.read_csv('test.csv')

In [ ]:
def preprocessing(words):


  f = open('nsmc_stopwords.txt')
  stop_word = f.read().split()

  review_text = re.sub("[^가-힣\\s]", "", words)
  word_review = mecab.morphs(words)
  word_review = [token for token in word_review if not token in stop_word and len(token) > 1]

  return word_review

In [ ]:
#okt.nouns를 tokenizer로 이용
tfidf = TfidfVectorizer(tokenizer=preprocessing,max_features=10000, min_df=5, max_df=0.5) 

X_train_tfidf = tfidf.fit_transform(train['overview'])
X_test_tfidf = tfidf.transform(test['overview'])

In [ ]:
encoder = LabelEncoder()
train_y = train['cat3']
train_y = encoder.fit_transform(train_y)

In [ ]:
kfold = StratifiedKFold(n_splits=10)

In [ ]:
# Random Forest 객체 생성
RFC = RandomForestClassifier(random_state=7)


# param_grid 설정
rf_param_grid = {"max_depth": [None],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}

# Grid Search 객체 생성
gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

# Grid Search 실행
gsRFC.fit(X_train_tfidf,train_y)

In [ ]:
# 최적의 파라미터 조합 출력
RFC_best = gsRFC.best_estimator_
print(RFC_best)

# 최고 점수(accuracy) 출력
print(gsRFC.best_score_)

In [ ]:
# AdaBoost 객체 생성
ADA = AdaBoostClassifier(random_state=7)

# param_grid 설정
ada_param_grid = {"n_estimators" :[1,20,30],
                  "algorithm" : ["SAMME","SAMME.R"],                  
              "learning_rate":  [0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

# Grid Search 객체 생성
gsADA = GridSearchCV(ADA, param_grid = ada_param_grid, cv=kfold, scoring="accuracy", verbose = 1)

# Grid Search 실행
gsADA.fit(X_train_tfidf, train_y)

In [ ]:
# 최적의 파라미터 조합 출력
ADA_best = gsADA.best_estimator_
print(ADA_best)

# 최고 점수(accuracy) 출력
print(gsADA.best_score_)

In [ ]:
#ExtraTrees 객체 생성
Ext = ExtraTreesClassifier(random_state=7)


# param_grid 설정
ex_param_grid = {"max_depth": [None],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}

# Grad Search 객체 생성
gsExt = GridSearchCV(Ext,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

# Grid Search 실행
gsExt.fit(X_train_tfidf,train_y)

In [ ]:
# 최적의 파라미터 조합 출력
Ext_best = gsExt.best_estimator_
print(Ext_best)

# 최고 점수(accuracy) 출력
print(gsExt.best_score_)

In [ ]:
#Support Vector 객체 생성
SVM = SVC(random_state=7, probability=True)

# param_grid 설정
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}

# Gradient boost 객체 생성
gsSVM = GridSearchCV(SVM,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

# Grid Search 실행
gsSVM.fit(X_train_tfidf,train_y)

In [ ]:
# 최적의 파라미터 조합 출력
SVM_best = gsSVM.best_estimator_
print(SVM_best)

# 최고 점수(accuracy) 출력
print(gsSVM.best_score_)

In [ ]:
#튜닝한 파라미터로 하드보팅
grid_soft = VotingClassifier(estimators = [('Random Forest', RFC_best),
                                           ('ExtraTrees', Ext_best),
                                           ('SVC', SVM_best),
                                           ('AdaBoost', ADA_best)], voting = 'soft')

grid_soft_cv = model_selection.cross_validate(grid_soft, X_train_tfidf, train_y, cv=10)

print("Soft voting on test set score mean: {:.2f}". format(grid_soft_cv['test_score'].mean() * 100))

In [ ]:
#튜닝한 파라미터로 하드보팅한 후 점수를 출력해보자.
grid_hard = VotingClassifier(estimators = [('Random Forest', RFC_best),
                                           ('ExtraTrees', Ext_best),
                                           ('SVC', SVM_best),
                                           ('AdaBoost', ADA_best)], voting = 'hard')

grid_hard_cv = model_selection.cross_validate(grid_hard, X_train_tfidf, train_y, cv=10)

print("Hard voting on test set score mean: {:.2f}". format(grid_hard_cv['test_score'].mean() * 100))

In [ ]:
cat3 = grid_soft.predict(X_test_tfidf)
cat3 = encoder.inverse_transform(cat3)

In [ ]:
test['cat3'] = cat3

In [ ]:
submission = test[['id','cat3']]
submission.to_csv('submission_ML.csv', index=False, encoding='utf-8')